In [ ]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random,datetime,os

import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros
from tensordict import TensorDict
from torchrl.data import TensorDictReplayBuffer,LazyMemmapStorage

## Environment
### Initialize Environment
In Mario, the environment consists of tubes, mushrooms and other components.

In [ ]:
# Initialize Super Mario environment
if gym.__version__<'0.26':
    env=gym_super_mario_bros.make("SuperMarioBros-1-1-v0",new_step_api=True)
else:
    env=gym_super_mario_bros.make("SuperMarioBros-1-1-v0",render_mode='rgb',apply_api_compatibility=True)

# Limit the action_space to
# 0. walk right
# 1.jump right
env = JoypadSpace(env,[["right"],["right","A"]])

env.reset()
next_state,reward,done,trunc,info=env.step(action=0)
print(f"{next_state.shape},\n {reward},\n {done},\n {info}")

## Preprocess Environment

In [ ]:
class SkipFrame(gym.wrappers):
    def __init__(self,env,skip):
        """Return only every 'skip'-th frame"""
        super().__init__(env)
        self.env=env
        self._skip=skip

    def step(self,action):
        """Repeat action, and sum reward"""
        total_reward=0.0
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs,r,done,trunc,info=self.env.step(action)
            total_reward+=r
            if done:
                break
        return obs,total_reward,done,trunc,info

In [ ]:
class GrayScaleObservation(gym.ObservationWrapper):
    def __init__(self,env):
        super().__init__(env)
        obs_shape=self.observation_space.shape[:2]
        self.observation_space=Box(low=0,high=255,shape=obs_shape,dtype=np.uint8)

    def permute_orientation(self,observation):
        # permute [H,W,C] array to [C,H,W] tensor
        observation=np.transpose(observation,(2,0,1))
        observation=torch.tensor(observation.copy(),dtype=torch.float)
        return observation

    def observation(self, observation):
        observation=self.permute_orientation(observation)
        transform=T.Grayscale()
        observation=transform(observation)
        return observation

In [ ]:
class ResizeObservation(gym.ObservationWrapper):
    def __init__(self,env,shape):
        super().__init__(env)
        if isinstance(shape,int):
            self.shape=(shape,shape)
        else:
            self.shape=tuple(shape)

        obs_shape=self.shape+self.observation_space.shape[:2]
        self.observation_space=Box(low=0,high=255,shape=obs_shape,dtype=np.uint8)

    def observation(self, observation):
        transforms=T.Compose(
            [T.Resize(self.shape,antialias=True),T.Normalize(0,255)]
        )
        observation=transforms(observation).squeeze(0)
        return observation

In [ ]:
# Apply Wrappers to environment
env=SkipFrame(env,skip=4)  # 帧跳过
env=GrayScaleObservation(env)  # 灰度化
env=ResizeObservation(env,shape=84)  # 调整大小
if gym.__version__<'0.26':
    env=FrameStack(env,num_stack=4,new_step_api=True)
else:
    env=FrameStack(env,num_stack=4)  # 帧堆栈


## Agent
We create a class `Mario` to represent our agent in the game. Mario should be able to:
* **Act** according to the optimal action policy based on the current state(of the environment).
* **Remember** experiences. Experience=(current state,current action,reward,next state).
* **Learn** a better action policy over time

In [ ]:
class Mario:
    def __init__(self):
        pass

    def act(self,state):
        """Given a state, choose an epsilon-greedy action"""
        pass

    def cache(self,experience):
        """Add the experience to memory"""
        pass

    def recall(self):
        """Sample experiences from memory"""
        pass

    def learn(self):
        """Update online action value (Q) function with a batch of experiences"""
        pass

## Act
For any given state, an agent can choose to do the most optimal action(**exploit**) or a random action(**explore**).

In [ ]:
# TODO
class Mario: